Import required functions

In [1]:
import math
from datetime import datetime
from math import inf

import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

Iris dataset

A function that takes an array of numbers and returns the lowest, like min function in numpy.

In [2]:
def average(values):
    if len(values) > 0:  # Ensure list is not empty
        sum = 0
        counter = 0
        for i in values:
            sum = sum + i
            counter = counter + 1
        return sum / len(values)  # Divide the total by the number of items
    return 0

Return the lowest item in a list.

In [3]:
def lowestDistance(lengthList):
    if len(lengthList) == 0:  # Check that the list is not empty
        return 0
    lowest = lengthList[0]
    item = 0
    for i in range(0, len(lengthList)):
        if lengthList[i] < lowest:
            lowest = lengthList[i]
            item = i
    return item

Returns the Euclidian distance between two points regardless of the number of dimensions (as long as they are the same).

In [4]:
def distanceBetween(x, y):
    square_differences = []
    for i in range(0, len(x)):
        square_differences.append(abs(y[i] - x[i]) * abs(y[i] - x[i]))  # Subtract and square coordinates
    sum = 0
    for i in square_differences:
        sum = sum + i
    distance = math.sqrt(sum)
    return distance

Returns the distance between the nearest neighbour to the given point.

In [5]:
def nearest_neighbour(lst, point):
    distanceList = []  # Create a new blank list to store distances
    for i in lst:
        distanceList.append(distanceBetween(i, point))  # Go through all points and return the distance to the sample
    for i in distanceList:
        if i == 0:  # Check that itself is not included in the distance list (we assume the point is not repeated)
            distanceList.remove(0)
    return lowestDistance(distanceList)

Return the index of a given item nearest items to the point

In [6]:
def nearestK(lst, point, k):
    outputList = []
    for i in range(0, k):
        outputList.append(nearest_neighbour(lst, point)) # Store the index of the nearest neighbour
        lst[nearest_neighbour(lst, point)] = [-math.inf] # Replace the element with the lowest value (removing it from the list)
    return outputList

# print(nearestK([[2, 3, 4111],[1,2,2]], [2, 3, 5], 1))
# print(nearest_neighbour([[2, 3, 411],[1,2,2]], [2, 3, 5]))

In [7]:
# Return the distance to the nearest sample that has the same label
def nearest_same(testing_label, training_labels, testing_coordinate, training_coordinates):
    training_labels_copy = np.array(training_labels.copy()).tolist() # Make copies so that the original list is not interfered with
    training_coordinates_copy = np.array(training_coordinates.copy()).tolist()
    while True:
        if len(training_coordinates_copy) == 0 or len(training_labels_copy) == 0:
            return inf # Prevent passing an empty list
        nearest = nearest_neighbour(training_coordinates_copy, testing_coordinate) # Continue to run the nearest neighbour algorithm until an item with the same label is found
        if training_labels_copy[nearest] == testing_label:
            if distanceBetween(training_coordinates_copy[nearest], testing_coordinate) == 0: # Check that the coordinate is not the same (the item itself) so that it can be removed
                training_coordinates_copy.pop(nearest)
                training_labels_copy.pop(nearest)
            else:
                return distanceBetween(training_coordinates_copy[nearest], testing_coordinate)
        else:
            training_labels_copy = np.delete(training_labels_copy, nearest, axis=0) # If there was no match then delete and continue looking
            training_coordinates_copy = np.delete(training_coordinates_copy, nearest, axis=0)


# Return the distance to the nearest sample that has a different label
def nearest_different(testing_label, training_labels, testing_coordinate, training_coordinates):
    training_labels_copy = training_labels.copy()
    training_coordinates_copy = training_coordinates.copy()

    while True:
        if len(training_coordinates_copy) == 0 or len(training_labels_copy) == 0:
            return inf # Prevent passing an empty list
        nearest = nearest_neighbour(training_coordinates_copy, testing_coordinate)
        if training_labels_copy[nearest] != testing_label: # Check for different label
            return distanceBetween(training_coordinates_copy[nearest], testing_coordinate) # Get the distance bwtween these two coordinates
        else:
            training_labels_copy = np.delete(training_labels_copy, nearest, axis=0) # If they mached to the same label then remove the coordinate and try again from the remaining ones
            training_coordinates_copy = np.delete(training_coordinates_copy, nearest, axis=0)

Return the conformity score for any given dataset and label.

In [8]:
def conformity(data, label, k):
    nearest_di = math.inf
    nearest_sa = math.inf
    shape = data.shape
    length = int(shape[0])
    for i in range(0, length): # Find the nearest neighbour
        cur_dis = distanceBetween(data[k], data[i])
        if cur_dis < nearest_di:
            if not i == k and label[k] == label[i]: # With the different label
                nearest_di = cur_dis
        if cur_dis < nearest_sa:
            if not i == k and not label[k] == label[i]: # With a same label
                nearest_sa = cur_dis
    if nearest_di == 0: # Check that it is not the same value (using zero distance check)
        if nearest_sa == 0:
            return 0
        return math.inf
    else:
        return nearest_sa / nearest_di

A function that returns the rank of a given item in a list.

In [9]:
def rnk(lst, value):
    rank = 0
    for i in range(0, len(lst)):
        if lst[i] < value:
            rank = rank + 1
    return rank + 1

Computing the p_value based on the set containing the ranks.

In [10]:
def p_val(ranks, training):
    n = len(training) + 1
    ranking = np.array(ranks) / n
    ranking = np.array(ranking).tolist()
    return ranking

Return the average of the false vales. Remove the correct values from the dataset and average the remaining values.

In [11]:
def averageFalse(testingSet, p_val):
    correctList = []
    for i in range(0, len(testingSet)):
        for j in np.unique(testingSet):
            if testingSet[i] != j:
                correctList.append(p_val[i][j])
    sum = 0
    count = 0
    for i in correctList:
        sum = sum + i
        count = count + 1
    return sum / count

Import the iris dataset and split it into the training and testing sets.

In [12]:
iris = load_iris()
iris_training_x, iris_testing_x, iris_training_y, iris_testing_y = train_test_split(iris['data'], iris['target'],
                                                                                    random_state=3001)

Running nearest neighbour algorithm on the iris dataset.

In [13]:
same = 0
total = 0
for i in range(0, len(iris_testing_x)):
    total = total + 1
    nn_i = nearest_neighbour(iris_training_x, iris_testing_x[i])
    if iris_training_y[nn_i] == iris_testing_y[i]:
        same = same + 1
print("Number of errors:", total - same)
print("Average correct rate:", same / total)
print("Average correct percentage:", (same / total) * 100, "%")
print("Average error rate:", (1 - (same / total)))
print("Average error percentage:", (1 - (same / total)) * 100, "%")

Number of errors: 4
Average correct rate: 0.8947368421052632
Average correct percentage: 89.47368421052632 %
Average error rate: 0.10526315789473684
Average error percentage: 10.526315789473683 %


In [14]:
def nearest_neighbour_k_iris(k):
    iris_training_x_copy = np.array(iris_training_x.copy())
    iris_training_y_copy = np.array(iris_training_y.copy())
    iris_testing_x_copy = np.array(iris_testing_x.copy())
    same = 0
    total = 0
    for i in range(0, len(iris_testing_x_copy)):
        tmp_lst = []
        nn_i = nearestK(iris_training_x_copy, iris_testing_x_copy[i], k)
        total = total + 1
        for j in nn_i:
            tmp_lst.append(iris_training_y_copy[j])
        for i in range(0, len(tmp_lst)):
            popular_label = np.bincount(tmp_lst).argmax()
            if popular_label == iris_training_y_copy[i]:
                same = same + 1
    print("Number of nearest neighbours:", k)
    print("Number of errors:", total - same)
    print("Average truth rate:", same / total)
    print("Average error rate:", (1 - (same / total)))
    print("Average error percentage:", (1 - (same / total)) * 100, "%")


nearest_neighbour_k_iris(2)

Number of nearest neighbours: 2
Number of errors: 16
Average truth rate: 0.5789473684210527
Average error rate: 0.42105263157894735
Average error percentage: 42.10526315789473 %


Get the conformal prediction for the iris dataset

In [15]:
x = datetime.now()
conformal_list_iris = []
lists = len(np.unique(iris_training_y))
for i in range(0, len(iris_testing_x)):
    temp_lst = []
    for j in np.unique(iris_training_y):
        if nearest_same(j, iris_training_y, iris_testing_x[i], iris_training_x) == 0:
            temp_lst.append(math.inf)
        else:
            temp_lst.append(
                nearest_different(j, iris_training_y, iris_testing_x[i], iris_training_x) / nearest_same(j,
                                                                                                         iris_training_y,
                                                                                                         iris_testing_x[
                                                                                                             i],
                                                                                                         iris_training_x))
    conformal_list_iris.append(temp_lst)
print(datetime.now() - x)

0:00:02.318138


Get the ranking of the conformity scores for each given label

In [16]:
conform = np.array(conformal_list_iris).T.tolist()
ranks = []
for i in conform:
    tmp_lst = []
    for j in i:
        tmp_lst.append(rnk(i, j))
    ranks.append(tmp_lst)
ranks = np.array(ranks).T.tolist()

Call the p_value function on the ranks to get the average p value for the iris dataset.

In [17]:
iris_pval = p_val(ranks, iris_training_y)
print("Average false P value", averageFalse(iris_testing_y, iris_pval))

Average false P value 0.11783884489986028


Ionosphere Dataset

Import the Ionosphere dataset and split it into the training set and the testing set and

In [18]:
X = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=np.arange(34))
y = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=34, dtype='int')
ion_training_x, ion_testing_x, ion_training_y, ion_testing_y = train_test_split(X, y, random_state=3001)
# ion_training_x, ion_testing_x, ion_training_y, ion_testing_y = train_test_split(X[:40], y[:40], random_state=3001)

Run the same nearest neighbour algorithm on the ionosphere dataset.

In [19]:
same = 0
total = 0
for i in range(0, len(ion_testing_x)):
    total = total + 1
    nn_i = nearest_neighbour(ion_training_x, ion_testing_x[i])
    if ion_training_y[nn_i] == ion_testing_y[i]:
        same = same + 1
print("Number of errors:", total - same)
print("Average correct rate:", same / total)
print("Average correct percentage:", (same / total) * 100, "%")
print("Average error rate:", (1 - (same / total)))
print("Average error rate:", (1 - (same / total)) * 100, "%")

Number of errors: 7
Average correct rate: 0.9204545454545454
Average correct percentage: 92.04545454545455 %
Average error rate: 0.07954545454545459
Average error rate: 7.954545454545459 %


In [20]:
x = datetime.now()


def nearest_neighbour_k_ion(k):
    ion_training_x_copy = np.array(ion_training_x.copy())
    ion_training_y_copy = np.array(ion_training_y.copy())
    ion_testing_x_copy = np.array(ion_testing_x.copy())
    same = 0
    total = 0
    for i in range(0, len(ion_testing_x_copy)):
        tmp_lst = []
        nn_i = nearestK(ion_training_x_copy, ion_testing_x_copy[i], k)
        total = total + 1
        for j in nn_i:
            tmp_lst.append(ion_training_y_copy[j] + 1)
        for i in range(0, len(tmp_lst)):
            popular_label = np.bincount(tmp_lst).argmax() + 1
            if popular_label == ion_training_y_copy[i]:
                same = same + 1
    print("Number of nearest neighbours:", k)
    print("Number of errors:", total - same)
    print("Average truth rate:", same / total)
    print("Average error percentage:", (1 - (same / total)) * 100, "%")


nearest_neighbour_k_ion(2)
print(datetime.now() - x)

Number of nearest neighbours: 2
Number of errors: 65
Average truth rate: 0.26136363636363635
Average error percentage: 73.86363636363636 %
0:00:02.018939


Run the conformity score on the ionosphere dataset and use this to calculate the average false p value.

In [21]:
x = datetime.now()
errors = 0
sum = 0
prediction = np.zeros(ion_testing_x.shape[0])
score = np.zeros(ion_training_x.shape[0] + 1)
p_score = np.zeros(len(np.unique(ion_training_y)))
for i in range(0, ion_testing_x.shape[0]):
    ax = np.vstack((ion_training_x, ion_testing_x[i]))
    for j in range(0, len(np.unique(ion_training_y))):
        ay = np.append(ion_training_y, j * 2 - 1)
        for k in range(ion_training_x.shape[0] + 1):
            score[k] = conformity(ax, ay, k)
        p_score[j] = average(score <= score[ion_training_x.shape[0]])
    prediction[i] = 2 * np.argmax(p_score) - 1
    if not prediction[i] == ion_testing_y[i]:
        errors = errors + 1
    sum = sum + p_score[0] + p_score[1] - p_score[ion_testing_y // 2]
print("Number of errors:", errors)
print("Average false P value:", errors / ion_testing_x.shape[0])
print(datetime.now() - x)

Number of errors: 6
Average false P value: 0.06818181818181818
0:04:33.693201


Convention for 0/0:
We want to record the distance to the nearest neighbour except if it is zero as they are not neighbours.
And we also do not want to compare the coordinate to itself.

Given a point with two equal distances, then we can return any.